In [1]:
import imp
#model = imp.load_source('model', '../modules/model.py')
svhn_images = imp.load_source('svhn_images', '../modules/svhn_images.py')

In [ ]:
tf_graph = model.getGraph()
session = model.init_interactive_session(tf_graph)

In [ ]:
model.load_saved_model('/flat2/saved_models/SVHN_3_digit_May7', session)

In [ ]:
#import matplotlib.image as mpimg
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches
#from matplotlib.patches import Rectangle
#from PIL import Image
#%matplotlib inline




# Download SVHN

In [2]:
data_kind = 'test'
print "Processing %s" % data_kind
svhn_images.download_and_extract_SVHN(data_kind, '/mnt/sandbox/raw')

outpath='/mnt/sandbox/%s' % data_kind

if not os.path.exists(outpath):
    os.mkdir(outpath)

svhn_images.separate_SVHN_images_labels('../data/SVHN/%s' % data_kind, \
                           '/mnt/sandbox/%s' %data_kind,
                           outpath)
print "Finished processing %s" % data_kind

Processing test
Finished downloading test
Extracting test
Finished extracting test
Starting image 1.png 
Starting image 1001.png 
Starting image 2001.png 
Starting image 3001.png 
Starting image 4001.png 
Starting image 5001.png 
Starting image 6001.png 
Starting image 7001.png 
Starting image 8001.png 
Starting image 9001.png 
Starting image 10001.png 
Starting image 11001.png 
Starting image 12001.png 
Starting image 13001.png 
Completed 13068 images
Finished processing test


In [ ]:
data_kind = 'train'
print "Processing %s" % data_kind
svhn_images.download_and_extract_SVHN(data_kind, '/flat2/sandbox')
svhn_images.separate_SVHN_images_labels('../data/SVHN/%s' % data_kind, \
                           '/flat2/sandbox/%s' % data_kind,
                           '/flat2/sandbox')
print "Finished processing %s" % data_kind